In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing ,svm ,tree,ensemble,linear_model
from sklearn.metrics import auc ,RocCurveDisplay 
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest ,SelectPercentile
from sklearn.decomposition import PCA

In [ ]:
# The FLAIR Dataset
df=pd.read_excel('FLAIR.xlsx')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Lets know how many values in colunm Grade
print("Grade        : \n",df["Grade"].value_counts())
print("\n")
print(df["Grade"].value_counts(normalize=True))

# plot.bar(title='') function is used to plot bargraph 
df["Grade"].value_counts().plot.bar(title="Grade")


In [ ]:
df["Grade"] = np.where(df["Grade"] == "G3", 0, 1)

In [ ]:
# Dropping the duplicate rows
duplicate_rows = df[df.duplicated()]
print("number of duplicate rows : ",duplicate_rows.shape)

# df.drop_duplicates(inplace=True)

In [ ]:
# To check missing values
print("SUM : \n",df.isnull().sum())
plt.figure(figsize=(8,8))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap="viridis")

# df["col"] = df["col"].fillna(df["col"].mean())
# df["col"] = df["col"].fillna(df["col"].interpolate())

In [ ]:
Xdata=df.iloc[:,1:]

In [ ]:
ydata=df.iloc[:,0].values

In [ ]:
scale=StandardScaler()
X=scale.fit_transform(Xdata)

In [ ]:
pca= PCA(n_components=128)
X=pca.fit_transform(X,y)

In [ ]:
df.shape

In [ ]:
from sklearn.feature_selection import SelectKBest
f=SelectKBest(k=15)
X_Kbest=f.fit_transform(X,y)

In [ ]:
from sklearn.feature_selection import SelectPercentile
f=SelectPercentile(percentile=30)
X_Percentile=f.fit_transform(X,y)

In [ ]:
# Standardization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_Kbest)
X_sts = scaler.transform(X_Kbest)

    
# Split into Input and Output Elements
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sts, y,stratify=y, test_size= 0.2,random_state=0)

print("X_train  = ",X_train.shape ," y_train = ", y_train.shape)
print("X_test   = ",X_test.shape ," y_test = ", y_test.shape)



In [ ]:
from sklearn.linear_model import LogisticRegression
LC=LogisticRegression()
LC.fit(X_train,y_train)
LC.score(X_train,y_train)

In [ ]:
LC.score(X_test,y_test)

In [ ]:
predictions = LC.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, predictions))

In [ ]:
cm = metrics.confusion_matrix(y_test, predictions)

sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
from sklearn.svm import SVC
Svc=SVC()
Svc.fit(X_train,y_train)
Svc.score(X_train,y_train)
Svc.score(X_test,y_test)

In [ ]:
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
predictions = Svc.predict(X_test)
print(accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

In [ ]:
cm = metrics.confusion_matrix(y_test, predictions)
sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTC=DecisionTreeClassifier()
DTC.fit(X_train,y_train)
DTC.score(X_train,y_train)

DTC.score(X_test,y_test)

In [ ]:
predictions = DTC.predict(X_test)

print(metrics.classification_report(y_test, predictions))

In [ ]:
cm = metrics.confusion_matrix(y_test, predictions)

sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
RFC.fit(X_train,y_train)
RFC.score(X_train,y_train)
RFC.score(X_test,y_test)

In [ ]:
predictions = RFC.predict(X_test)
print(metrics.classification_report(y_test, predictions))

In [ ]:
cm = metrics.confusion_matrix(y_test, predictions)

sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
# Run classifier with cross-validation and plot ROC curves
for num in range(10,51,5):
    print('percentile of SelectPercentile is ',num)
    f=SelectPercentile(percentile=num)
    X=f.fit_transform(Xdata,y)
    cv = StratifiedKFold(n_splits=4)
    classifier = svm.SVC(C=1.0,kernel='poly')

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots()
    for i, (train, test) in enumerate(cv.split(X, y)):

        classifier.fit(X[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="Receiver operating characteristic example",
    )
    ax.legend(loc="lower right")
    plt.show()

In [1]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
print(kmeans.labels_)
print(kmeans.predict([[0, 0], [12, 3]]))
print(kmeans.cluster_centers_)

[1 1 1 0 0 0]
[1 0]
[[10.  2.]
 [ 1.  2.]]
